In [1]:
%cd "C:\Users\andre\Downloads\Data"

C:\Users\andre\Downloads\Data


In [6]:
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import pyarrow as pa
from tqdm.auto import tqdm

In [3]:
source_path = Path('data/london_smart_meters/')
block_data_path = source_path / 'hhblock_dataset' / 'hhblock_dataset'

In [4]:
# Read with pandas
block_1_pd = pd.read_csv(block_data_path / 'block_0.csv', parse_dates=False)

# Read with polars
block_1_pl = pl.read_csv(block_data_path / 'block_0.csv', try_parse_dates=False)

In [ ]:
%timeit block_1_pd = pd.read_csv(block_data_path / 'block_0.csv', parse_dates=False)

74.1 ms ± 838 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit block_1_pl = pl.read_csv(block_data_path / 'block_0.csv', try_parse_dates=False)

8.47 ms ± 209 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
block_1_pl.head()

LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,hh_8,hh_9,hh_10,hh_11,hh_12,hh_13,hh_14,hh_15,hh_16,hh_17,hh_18,hh_19,hh_20,hh_21,hh_22,hh_23,hh_24,hh_25,hh_26,hh_27,hh_28,hh_29,hh_30,hh_31,hh_32,hh_33,hh_34,hh_35,hh_36,hh_37,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""MAC000002""","""2012-10-13""",0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,0.167,0.109,0.168,0.107,0.166,0.117,0.157,0.126,0.146,0.106,0.135,0.191,0.915,0.933,0.122,0.138,0.076,0.133,0.076,0.133,0.085,0.263,0.134,0.235,0.124,0.184,0.23,0.176,0.388,0.26,0.918,0.278,0.267,0.239,0.23,0.233,0.235,0.188,0.259,0.25
"""MAC000002""","""2012-10-14""",0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,0.12,0.079,0.121,0.075,0.124,0.073,0.125,0.07,0.13,0.108,0.196,0.346,0.524,0.076,0.129,0.667,0.23,0.22,0.163,0.091,0.17,0.11,0.11,0.121,0.099,0.157,0.093,0.371,0.386,1.085,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.21,0.278,0.159
"""MAC000002""","""2012-10-15""",0.192,0.097,0.141,0.083,0.132,0.07,0.13,0.074,0.124,0.078,0.118,0.082,0.112,0.087,0.106,0.14,0.12,1.075,0.146,0.123,0.082,0.127,0.077,0.551,0.149,0.129,0.075,0.13,0.075,0.129,0.075,0.128,0.166,0.194,0.695,0.26,0.227,0.255,1.164,0.249,0.225,0.258,0.26,0.334,0.299,0.236,0.241,0.237
"""MAC000002""","""2012-10-16""",0.237,0.237,0.193,0.118,0.098,0.107,0.094,0.109,0.091,0.105,0.091,0.104,0.092,0.103,0.093,0.101,0.144,0.1,0.408,0.102,0.1,0.116,0.354,0.146,0.19,0.991,0.31,0.121,0.113,0.094,0.119,0.087,0.13,0.238,0.204,0.284,0.447,0.266,0.966,0.172,0.192,0.228,0.203,0.211,0.188,0.213,0.157,0.202
"""MAC000002""","""2012-10-17""",0.157,0.211,0.155,0.169,0.101,0.117,0.084,0.118,0.08,0.119,0.075,0.123,0.071,0.126,0.067,0.124,0.118,0.132,0.358,0.628,0.784,0.681,0.749,0.593,0.502,0.115,0.113,0.092,0.124,0.084,0.125,0.078,0.136,0.227,0.207,0.141,0.258,0.217,0.223,0.075,0.23,0.208,0.265,0.377,0.327,0.277,0.288,0.256


In [37]:
block_1_pd.head()

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000002,2012-10-13,0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,...,0.918,0.278,0.267,0.239,0.230,0.233,0.235,0.188,0.259,0.250
1,MAC000002,2012-10-14,0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,...,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.210,0.278,0.159
2,MAC000002,2012-10-15,0.192,0.097,0.141,0.083,0.132,0.070,0.130,0.074,...,1.164,0.249,0.225,0.258,0.260,0.334,0.299,0.236,0.241,0.237
3,MAC000002,2012-10-16,0.237,0.237,0.193,0.118,0.098,0.107,0.094,0.109,...,0.966,0.172,0.192,0.228,0.203,0.211,0.188,0.213,0.157,0.202
4,MAC000002,2012-10-17,0.157,0.211,0.155,0.169,0.101,0.117,0.084,0.118,...,0.223,0.075,0.230,0.208,0.265,0.377,0.327,0.277,0.288,0.256


In [7]:
# convert pandas df date column from object to datetime
block_1_pd['day'] = pd.to_datetime(block_1_pd['day'], format='%Y-%m-%d')

# convert polars df date column from object to datetime
block_1_pl = block_1_pl.with_columns(
        pl.col('day').str.to_datetime(format='%Y-%m-%d').alias('day')
)

In [8]:
# check pandas end dates of time series
block_1_pd.groupby('LCLid')['day'].sample(5)

407     2013-11-28
261     2013-07-05
19      2012-11-01
206     2013-05-11
120     2013-02-14
           ...    
25110   2013-09-04
24965   2012-12-02
25280   2014-02-22
25100   2013-08-25
25208   2013-12-11
Name: day, Length: 250, dtype: datetime64[ns]

In [11]:
# check polars end dates of time series
block_1_pl.group_by(pl.col('LCLid')).agg(pl.col('day').max()).sample(5)

LCLid,day
str,datetime[μs]
"""MAC004387""",2014-02-27 00:00:00
"""MAC003482""",2014-02-27 00:00:00
"""MAC000246""",2014-02-27 00:00:00
"""MAC003874""",2014-02-27 00:00:00
"""MAC003686""",2014-02-27 00:00:00


In [ ]:
# Check max date across all blocks with Pandas

max_date = None

for f in tqdm(block_data_path.glob('*.csv')):
    df = pd.read_csv(f, parse_dates=False)
    df['day'] = pd.to_datetime(df['day'], format='%Y-%m-%d')
    if max_date is None:
        max_date = df['day'].max()
    elif df['day'].max() > max_date:
        max_date = df['day'].max()
        
print(f'max date across all blocks is: {max_date}')

112it [00:14,  7.49it/s]

max date across all blocks is: 2014-02-27 00:00:00


In [ ]:
# Check max date across all blocks with Polars

max_date = None

for f in tqdm(block_data_path.glob('*.csv')):
    df = pl.read_csv(f, try_parse_dates=False)
    df = df.with_columns(
        pl.col('day').str.to_datetime(format='%Y-%m-%d')
        )
    if max_date is None:
        max_date = df.select(pl.col('day')).max().item()
    elif df.select(pl.col('day')).max().item() > max_date:
        max_date = df.select(pl.col('day')).max().item()

print(f'max date across all blocks is: {max_date}')

112it [00:01, 57.11it/s]

max date across all blocks is: 2014-02-27 00:00:00
